In [2]:
import numpy as np
from pymol import cmd

In [18]:
import numpy as np

v1 = np.array([1, 2, 3])
v2 = np.array([4, 5, 6])

dot_product = np.dot(v1, v2)
magnitude_v1 = np.linalg.norm(v1)
magnitude_v2 = np.linalg.norm(v2)
print(magnitude_v1,magnitude_v2)
print(np.sqrt(np.sum(v1**2)))
angle_radians = np.arccos(dot_product / (magnitude_v1 * magnitude_v2))
angle_degrees = np.degrees(angle_radians)

print(f"The angle between the two vectors is {angle_degrees} degrees.")

import numpy as np

value = -1/3
angle_radians = np.arccos(-1/3)

print(f"The arccosine of {value} is {angle_radians} radians.")
print(np.degrees(angle_radians))

3.7416573867739413 8.774964387392123
3.7416573867739413
The angle between the two vectors is 12.933154491899135 degrees.
The arccosine of -0.3333333333333333 is 1.9106332362490186 radians.
109.47122063449069


In [55]:
import numpy as np

def normalize_vector(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm

def calculate_rotation_axis(v1, v2):
    v1_normalized = normalize_vector(v1)
    v2_normalized = normalize_vector(v2)
    axis = np.cross(v1_normalized, v2_normalized)
    return normalize_vector(axis)

def calculate_rotation_angle(v1, v2):
    v1_normalized = normalize_vector(v1)
    v2_normalized = normalize_vector(v2)
    dot_product = np.dot(v1_normalized, v2_normalized)
    return np.arccos(dot_product)

def create_rotation_matrix(axis, angle):
    c = np.cos(angle)
    s = np.sin(angle)
    t = 1 - c
    x, y, z = axis
    return np.array([
        [t * x**2 + c, t * x * y - s * z, t * x * z + s * y],
        [t * x * y + s * z, t * y**2 + c, t * y * z - s * x],
        [t * x * z - s * y, t * y * z + s * x, t * z**2 + c]
    ])

def rotate_vector(vector, rotation_matrix):
    return np.dot(rotation_matrix, vector)

A1 = np.array([0,0,1])
A2 = np.array([np.sqrt(8/9),0,-1/3])
print(A1,A2)
d = 1.5336
axis = calculate_rotation_axis(A1,A2)
angle = calculate_rotation_angle(A1,A2)
# print(axis,angle)
middle_vector = rotate_vector(A1,create_rotation_matrix(axis,angle/2))
angle = np.arccos(-1/3)
A1_adjusted = rotate_vector(middle_vector,create_rotation_matrix(axis,-angle/2))
A2_adjusted = rotate_vector(middle_vector,create_rotation_matrix(axis,angle/2))
angle = np.pi*(2/3)
A1 = rotate_vector(A2_adjusted,create_rotation_matrix(A1_adjusted,angle))
print("akjklfsjl",A1,A2)
# A1 = rotate_vector(middle_vector,create_rotation_matrix(axis,np.pi))
# axis = calculate_rotation_axis(A1,middle_vector)
# A1 = rotate_vector(middle_vector,create_rotation_matrix(axis,-angle/2))
# A2 = rotate_vector(middle_vector,create_rotation_matrix(axis,angle/2))
x = normalize_vector(np.cross(A2,A1))
axis = calculate_rotation_axis(middle_vector,x)
# angle = np.arccos(-1/3)
angle = np.pi*(2/3)
A1 = rotate_vector(middle_vector,create_rotation_matrix(axis,angle))
print("ahjwjfjwkl",x,middle_vector,A1,np.linalg.norm(middle_vector),axis,angle)

axis = np.array([0,0,1])
print(rotate_vector(A2,create_rotation_matrix(axis,angle)))


middle_rotated = np.array([np.sqrt(8/9),0,-1/3])
axis = calculate_rotation_axis(middle_rotated,middle_vector)
angle = calculate_rotation_angle(middle_rotated,middle_vector)
A1_rotated = np.array([0,0,1])
# print(axis,angle)
print(create_rotation_matrix(axis,angle))
A2_rotated = rotate_vector(A2,create_rotation_matrix(axis,angle)) #check angle
print([-np.sqrt(2/9),-np.sqrt(2/3),-(1/3)])
print("hej",A1_rotated,A2_rotated)
# A1_norm = (A1/(np.sqrt(np.sum(A1**2))))*d # normalize to 1.5336
# A2_norm = (A2/(np.sqrt(np.sum(A2**2))))*d # normalize to 1.5336

axis = calculate_rotation_axis(A1,A2)
angle = np.pi/2
virtual_center = rotate_vector(A1,create_rotation_matrix(axis,angle)) #check angle
d = np.sqrt(np.sum(A1**2))




v1 = np.array([1,0,0])
v2 = np.array([0,1,0])

axis = calculate_rotation_axis(v1, v2)
angle = calculate_rotation_angle(v1, v2)
rotation_matrix = create_rotation_matrix(axis, angle)
rotated_v1 = rotate_vector(v1, rotation_matrix)

print(axis,angle,rotation_matrix)
print("Original Vector v1:", v1)
print("Rotated Vector v1:", rotated_v1)

A1 = np.array([1,0,0])
A2 = np.array([0,1,0])

axis = calculate_rotation_axis(A1,A2)
angle = calculate_rotation_angle(A1, A2)
rotation_matrix = create_rotation_matrix(axis,angle/2)
middel = rotate_vector(A1,rotation_matrix)
print(middel)
rotation_matrix = create_rotation_matrix(axis,angle/2)
rotation_matrix2 = create_rotation_matrix(axis,-angle/2)
hej = rotate_vector(middel,rotation_matrix)
hej2 = rotate_vector(middel,rotation_matrix2)
print(hej,hej2)

[0 0 1] [ 0.94280904  0.         -0.33333333]
akjklfsjl [-0.47140452  0.81649658 -0.33333333] [ 0.94280904  0.         -0.33333333]
ahjwjfjwkl [0.28867513 0.5        0.81649658] [0.81649658 0.         0.57735027] [-0.76180168  0.61237244  0.21132487] 1.0 [-0.40824829 -0.70710678  0.57735027] 2.0943951023931953
[-0.47140452  0.81649658 -0.33333333]
[[ 0.57735027 -0.         -0.81649658]
 [ 0.          1.         -0.        ]
 [ 0.81649658  0.          0.57735027]]
[-0.4714045207910317, -0.816496580927726, -0.3333333333333333]
hej [0 0 1] [0.81649658 0.         0.57735027]
[0. 0. 1.] 1.5707963267948966 [[ 6.123234e-17 -1.000000e+00  0.000000e+00]
 [ 1.000000e+00  6.123234e-17  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  1.000000e+00]]
Original Vector v1: [1 0 0]
Rotated Vector v1: [6.123234e-17 1.000000e+00 0.000000e+00]
[0.70710678 0.70710678 0.        ]
[1.79380389e-16 1.00000000e+00 0.00000000e+00] [1.00000000e+00 1.01465364e-17 0.00000000e+00]


In [46]:
import numpy as np

# Coordinates of the two known vertices
vertex1 = np.array([0,0,1])
vertex2 = np.array([np.sqrt(8/9),0,-1/3])

# A1 = np.array([0,0,1])
# A2 = np.array([np.sqrt(8/9),0,-1/3])

# Calculate the midpoint of the two known vertices
midpoint = (vertex1 + vertex2) / 2

# Calculate the distance from the centroid (0, 0, 0) to the midpoint
distance = np.linalg.norm(midpoint)

# Calculate the height of the tetrahedron (distance from centroid to the center of a face)
height = np.sqrt(2/3) * distance

# Calculate the coordinates of the remaining two vertices
vertex3 = midpoint + np.array([0, 0, height])
vertex4 = midpoint - np.array([0, 0, height])
print(distance)

print("Vertex 1:", vertex1)
print("Vertex 2:", vertex2)
print("Vertex 3:", vertex3)
print("Vertex 4:", vertex4)

0.5773502691896258
Vertex 1: [0 0 1]
Vertex 2: [ 0.94280904  0.         -0.33333333]
Vertex 3: [0.47140452 0.         0.80473785]
Vertex 4: [ 0.47140452  0.         -0.13807119]


In [1]:
import foldseek_virtual_center

In [1]:
from pymol import cmd
from scipy.spatial import cKDTree

In [2]:
structure_path = "../SP/SP3.pdb"
cmd.load(structure_path)
structure = structure_path.split("/")[-1].split(".")[0]
model = cmd.get_model(structure)
model_CA = cmd.get_model(structure+" and name CA and chain A and not HETATM")

In [3]:
import numpy as np

In [6]:
kj = []
for x in range(439):
    kj.append(0.4)

In [5]:
hotspot_finder.run(kj,[structure],[0.3,0.7],10)

[[-31.27161837 -27.45453314   4.18267091]
 [-35.27153395 -26.87579164   3.07282711]
 [-35.07051204 -31.20307277   6.7700365 ]
 ...
 [-21.07395901 -11.63934406   8.83837364]
 [-26.61625864 -11.13774644   7.32226127]
 [-25.94232905  -5.84727377   9.74118384]]


In [4]:
import hotspot_finder
k = hotspot_finder.foldseek_virtual_center(model,model_CA)
np.shape(k)
print(k)

1 439 A
1 439 A
1 439 A
1 439 A
1 439 A
1 439 A
1 439 A
1 439 A
1 439 A
1 439 A
1 439 A
1 439 A
1 439 A
1 439 A
1 439 A
1 439 A
1 439 A
1 439 A
1 439 A
2 439 A
2 439 A
2 439 A
2 439 A
2 439 A
2 439 A
2 439 A
2 439 A
2 439 A
2 439 A
2 439 A
3 439 A
3 439 A
3 439 A
3 439 A
3 439 A
3 439 A
3 439 A
3 439 A
3 439 A
3 439 A
3 439 A
3 439 A
3 439 A
3 439 A
3 439 A
4 439 A
4 439 A
4 439 A
4 439 A
4 439 A
4 439 A
4 439 A
4 439 A
4 439 A
4 439 A
4 439 A
4 439 A
4 439 A
4 439 A
4 439 A
4 439 A
4 439 A
4 439 A
4 439 A
5 439 A
5 439 A
5 439 A
5 439 A
5 439 A
5 439 A
5 439 A
5 439 A
5 439 A
5 439 A
5 439 A
6 439 A
6 439 A
6 439 A
6 439 A
6 439 A
6 439 A
6 439 A
6 439 A
6 439 A
6 439 A
7 439 A
7 439 A
7 439 A
7 439 A
7 439 A
7 439 A
7 439 A
7 439 A
7 439 A
7 439 A
7 439 A
7 439 A
7 439 A
7 439 A
7 439 A
7 439 A
7 439 A
7 439 A
7 439 A
8 439 A
8 439 A
8 439 A
8 439 A
8 439 A
8 439 A
8 439 A
8 439 A
8 439 A
8 439 A
8 439 A
8 439 A
8 439 A
8 439 A
8 439 A
9 439 A
9 439 A
9 439 A
9 439 A
9 439 A
9 439 A


(439, 3)

In [16]:
kage = []
m = cmd.get_model(structure+" and chain A and not HETATM")
print([x.resi for x in m.atom])
print(np.max(np.array([int(x.resi) for x in m.atom])))
for x in model.atom:
    if x.chain == "A":
        kage.append(x)
len(kage)

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '8', '8', '8', '8', '8', '8', '8', '8', '8', '8', '8', '8', '8', '8', '8', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '12', '12', '12', '12', '12', '12', '12', '12', '12', '12', '13', '13', '13', '13', '13', '13', '13', '13', '13', '13', '13', '13', '13', '13', '13', '13', '13', '13', '13', '14', '14', '14', '14', '14', '14', '14'

6390

In [8]:
ref_coord = model.atom[56].coord
ref_coord = [4,6,8]
kd = cKDTree([atom.coord for atom in model.atom])
closest_pair = kd.query(ref_coord)
print(closest_pair)

(1.3721094048312612, 2839)


In [4]:
model.atom[45]52

In [58]:
round(-234.23445325,3)

-234.234

In [93]:
foldseek_virtual_center.foldseek_virtual_center

<function foldseek_virtual_center.foldseek_virtual_center(model)>

In [2]:
import os
for x in os.listdir("thermoDB"):
    os.system(f"mv thermoDB/{x} thermoDB/thermoDB{x.split('v2_opT-min70')[-1]}")

In [5]:
test = [1,2,3,4,5,6,7,8,9,10]
for x in range(13):
    print(test[x%10])

1
2
3
4
5
6
7
8
9
10
1
2
3


In [7]:
kage = [
    [206,87,87],
    [106,120,232],
    [101,213,119],
    [208,98,211],
    
    [231,163,116],
    [104,227,178],
    [69,180,214],
    [175,143,210],
    [229,174,196],
    [224,202,107],
    [228,228,228]      
    ]
iii = []
for ele in kage:
    iii.append([round(x/255,2) for x in ele])
print(iii)

[[0.81, 0.34, 0.34], [0.42, 0.47, 0.91], [0.4, 0.84, 0.47], [0.82, 0.38, 0.83], [0.91, 0.64, 0.45], [0.41, 0.89, 0.7], [0.27, 0.71, 0.84], [0.69, 0.56, 0.82], [0.9, 0.68, 0.77], [0.88, 0.79, 0.42], [0.89, 0.89, 0.89]]


In [71]:
len(str(-123.345))

8

In [4]:
import foldseek_virtual_center
# def add_space(number,max_length,decimaler):
#     number = str(number)
#     if len(number.split(".")) > 1:
#         while len(number.split(".")[-1]) != decimaler:
#             number += "0"
# def add_space(number,max_length,decimaler):
#     number = str(number)
#     if len(number.split(".")) > 1:
#         while len(number.split(".")[-1]) != decimaler:
#             number += "0"
#     space = " "
#     for x in range(int(max_length-len(str(number)))):
#         space += " "
#     return space + str(number)

# def virtual_center_to_pdb(virtual_centers,outfilename,structure_name):
#     model = cmd.get_model(structure_name+" and name CA").atom
#     with open(outfilename,"w") as out:
#         out.write("MODEL\t1\n")
#         for i,v in enumerate(virtual_centers):
#             resi = str(i+1)
#             resi = add_space(resi,6,0)
#             v0 = add_space(round(v[0],3),7,3)
#             v1 = add_space(round(v[1],3),7,3)
#             v2 = add_space(round(v[2],3),7,3)
#             out.write(f"ATOM{resi}  CA  {model[i].resn} A{resi[3:]}    {v0}{v1}{v2}\t1.00 98.00           C\n")
#     space = " "
#     for x in range(int(max_length-len(str(number)))):
#         space += " "
#     return space + str(number)
# kage = cmd.get_model(structure+" and name CA").atom
vi = foldseek_virtual_center.foldseek_virtual_center(model)
foldseek_virtual_center.virtual_center_to_pdb(vi,"tmp/virtual_SP1.pdb",structure)



In [ ]:
MODEL     1                                                                     
ATOM      1  CA   ALA A   1      -9.758  -6.578  16.438  1.00 58.84           C
          s               s               koord           no  100.00          C

In [ ]:
foldseek_virtual_center(model)

In [13]:
print(model.atom.coord)

<bound method Base.list of <chempy.models.Indexed object at 0x7fda7b2eceb0>>
